In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt
import csv
import sqlite3
import nltk
from nltk.stem import WordNetLemmatizer as wnl

sys.path.append('./data')
sys.path.append('./models')

from process_data import *
# from train_classifier import load_data

nltk.download('words'); # English word corpus for filtering data
nltk.download('wordnet');

# Project Overview
## ETL Pipeline: process_data.py
In a Python script, process_data.py, write a data cleaning pipeline that:
- Loads the messages and categories datasets
- Merges the two datasets
- Cleans the data
- Stores it in a SQLite database

## ML pipeline: train_classifier.py
In a Python script, train_classifier.py, write a machine learning pipeline that:
- Loads data from the SQLite database
- Splits the dataset into training and test sets
- Builds a text processing and machine learning pipeline
- Trains and tunes a model using GridSearchCV
- Outputs results on the test set
- Exports the final model as a pickle file

## Flask Web App
We are providing much of the flask web app for you, but feel free to add extra features depending on your knowledge of flask, html, css and javascript. For this part, you'll need to:
- Modify file paths for database and model as needed
- Add data visualizations using Plotly in the web app. One example is provided for you.

## Github and Code Quality
Your project will also be graded based on the following:
- Use of Git and Github
- Strong documentation
- Clean and modular code
- Follow the [RUBRIC](https://learn.udacity.com/nanodegrees/nd025/parts/cd0018/lessons/e692c8ed-b713-464b-95ac-72d93a35b4fc/concepts/e692c8ed-b713-464b-95ac-72d93a35b4fc-project-rubric) when you work on your project to assure you meet all of the necessary criteria for developing the pipelines and web app.

# process_data.py

In [ ]:
DATA_DIR = './data'
categories_filepath = os.path.join(DATA_DIR, 'disaster_categories.csv')
messages_filepath = os.path.join(DATA_DIR, 'disaster_messages.csv')
database_filepath = os.path.join(DATA_DIR, 'DisasterResponse.db')

df = clean_data(load_data(messages_filepath, categories_filepath))
save_data(df, database_filepath=database_filepath)

In [ ]:
# make sure it works
conn = sqlite3.connect(database_filepath)

cur = conn.cursor()

test = pd.read_sql(f'SELECT * from project2', con=conn, index_col='id')
conn.close()

test.head()

# train_classifier.py

# app.py

# CRISP-DM Flow

## Business Understanding
- The purpose of this project is to identify an efficient way to model emergency messages to determine which are most informative for making a quick disaster response decision. This is important as resources for disaster response are limited and false positives can prevent those resources from being deployed where they are needed most.

## Data Understanding
- The dataset is raw, containing an identifier along with communication information that can contain an English 'message', a non-English 'original' messge, and a 'genre' (type of communication). This insformation is provided in CSV form.
- The content of each row is inconsistent - for example, some of the lines contain more than 4 commas (which would cleanly correspond to the 4 expected fields), some contain consecutive commas (where one of the messages is missing), and some contain quotation marks separating the English and non-English messages.
- There is also categorical information stored in a separate CSV, which provides some informative identifiers that are pre-associated with each message (and link by an 'id' field in each set of data).
    - This CSV data is further subdivided into key/value pairs separated by a semicolon, e.g. 'id,key1-value1;key2-value2; ...' etc.
- **load_data.py** loads the raw message and category information, first splits out the 'id' and the 'categories' key/value pairs, then splits the key/value pairs into columns of data with integer data, and joins each dataset (on 'id') and passes that joined dataframe along for cleaning in the next phase.

## Data Preparation
- **clean_data.py** attempts to separate the Enlish and non-English components of the raw nessage, and stores the processed English-only message as a new field in the data, allowing it to be more easily processed in the machine learning pipeline.
- It also converts the 'genre' field into a set of dummies with boolean values (n-1 categories).

In [ ]:
# why does 'related' have a max value of 2?
df[[label for label, dtype in df.dtypes.items() if dtype in [int, np.int64]]].describe().T

In [ ]:
np.int64

## Data Modeling

In [ ]:
pd.Int64Dtype

## Result Evaluation

## Deployment